In [1]:
from transformers import AutoTokenizer, CLIPTextModelWithProjection, CLIPVisionModelWithProjection,  AutoProcessor
from PIL import Image
import requests
import torch

In [2]:
model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")


Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPTextModelWithProjection: ['vision_model.encoder.layers.2.self_attn.out_proj.bias', 'vision_model.encoder.layers.2.mlp.fc2.weight', 'vision_model.encoder.layers.1.mlp.fc1.weight', 'vision_model.encoder.layers.9.self_attn.q_proj.weight', 'vision_model.encoder.layers.1.layer_norm2.weight', 'vision_model.encoder.layers.2.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.mlp.fc2.weight', 'vision_model.encoder.layers.11.mlp.fc2.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.5.layer_norm1.weight', 'vision_model.encoder.layers.6.mlp.fc2.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.weight', 'vision_model.encoder.layers.3.self_attn.out_proj.weight', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.9.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.self_attn.v_proj.bias', 'vision_model.e

In [3]:
inputs = tokenizer(["a pair of Kittens sleeping","a pair of Kittens sleeping", "a photo of a dog"], padding=True, return_tensors="pt")

In [4]:
outputs = model(**inputs)

In [5]:
outputs

CLIPTextModelOutput(text_embeds=tensor([[ 0.0786, -0.1164, -0.4900,  ..., -0.1514, -0.4523, -0.2023],
        [ 0.0786, -0.1164, -0.4900,  ..., -0.1514, -0.4523, -0.2023],
        [ 0.0932,  0.2764, -0.4137,  ..., -0.5852, -0.2590,  0.1193]],
       grad_fn=<MmBackward0>), last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 0.2446, -0.0337,  0.0675,  ..., -0.0108, -0.1678, -2.1158],
         ...,
         [ 0.3420, -0.5756,  1.2779,  ..., -0.0484,  0.5701, -1.7490],
         [ 0.5918, -0.3644,  0.1242,  ..., -0.2798,  1.1070, -1.0061],
         [-0.3823,  0.4144,  1.6134,  ...,  0.0771,  0.5741, -1.9464]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 0.2446, -0.0337,  0.0675,  ..., -0.0108, -0.1678, -2.1158],
         ...,
         [ 0.3420, -0.5756,  1.2779,  .

In [6]:
text_embeds = outputs.text_embeds

In [7]:
np.array(text_embeds[1].tolist())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 np.array(text_embeds[1].tolist())                                                            │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'np' is not defined

In [8]:
model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.11.self_attn.q_proj.bias', 'text_model.encoder.layers.11.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.v_proj.weight', 'text_model.encoder.layers.0.layer_norm1.bias', 'text_model.encoder.layers.9.self_attn.q_proj.weight', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.bias', 'text_model.encoder.layers.6.mlp.fc2.weight', 'text_model.encoder.layers.2.self_attn.v_proj.bias', 'text_model.encoder.layers.0.layer_norm2.bias', 'text_model.encoder.layers.6.self_attn.k_proj.weight', 'text_model.encoder.layers.6.layer_norm2.weight', 'text_model.encoder.layers.0.mlp.fc2.bias', 'text_model.encoder.layers.7.layer_norm1.bias', 'text_model.encoder.layers.9.mlp.fc2.bias', 'text_model.encoder.layers.8.layer_norm2.weight', 'text_model.encoder.layers.9.self_attn.k_proj.weight', 'text_m

In [9]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [10]:
inputs = processor(images=image, return_tensors="pt")

In [11]:
outputs = model(**inputs)

In [12]:
image_embeds = outputs.image_embeds

In [13]:
len(image_embeds)

1

In [14]:
len(image_embeds.tolist()[0])

512

In [15]:
import numpy as np

In [16]:
img_arr = np.array(image_embeds.tolist()[0])

In [17]:
text_arr = np.array(text_embeds[0].tolist())

In [18]:
np.dot(img_arr, text_arr)

26.27812504571261

In [19]:
from numpy.linalg import norm

In [20]:
cos_sim = np.dot(img_arr, text_arr)/(norm(img_arr)*norm(text_arr))

In [21]:
cos_sim

0.2940507353334896

In [22]:
similarity_score = torch.nn.functional.cosine_similarity(text_embeds[1], text_embeds[2])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 similarity_score = torch.nn.functional.cosine_similarity(text_embeds[1], text_embeds[2])     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [23]:
similarity_score

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 similarity_score                                                                             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'similarity_score' is not defined

In [24]:
text_embeds[0]

tensor([ 7.8639e-02, -1.1640e-01, -4.8996e-01, -1.8113e-02, -1.2069e-01,
         2.7287e-01, -3.5920e-01, -5.1525e-01, -3.8022e-01, -1.5557e-01,
         1.0042e-01, -2.6965e-01, -2.7772e-01, -6.5565e-01, -5.7275e-01,
         2.4773e-01, -2.5985e-01, -1.3073e-01,  1.5946e-01,  6.2904e-01,
        -1.1369e-01, -7.7783e-02,  1.0770e-01, -1.3284e-01,  3.0105e-02,
         1.3388e-01,  1.5950e-01,  7.7331e-01, -1.3675e-01, -6.5382e-02,
         8.0658e-02, -8.6160e-02,  1.6838e-01,  3.9098e-01, -9.9478e-01,
         5.9946e-03,  3.2895e-01,  3.1184e-01,  1.0530e-01,  9.1251e-02,
         2.5212e-01, -1.1359e-01, -6.3757e-02, -5.5682e-02,  3.2144e-01,
         5.0882e-02, -4.5993e-02, -1.5483e-01, -1.1238e-01, -1.1533e-01,
         4.2757e-01, -2.0133e-02,  3.1974e-01, -1.8372e-01, -1.1654e-01,
         7.1383e-03, -6.8467e-01,  2.2001e-01, -1.3863e-02, -2.9394e-01,
        -1.1392e-02, -1.2948e-01, -1.7851e-01, -3.4290e-01,  2.3019e-01,
        -5.4519e-02, -5.7900e-03,  2.7436e-01, -3.8